In [2]:
import gurobipy as gb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import random
import function_library_assignment_2 as fnc

%load_ext autoreload
%autoreload 2

plt.rcParams['font.size']=12
plt.rcParams['font.family']='serif'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False  
plt.rcParams['axes.spines.bottom'] = True     
plt.rcParams["axes.grid"] =True
plt.rcParams['grid.linestyle'] = '-.' 
plt.rcParams['grid.linewidth'] = 0.4
plt.rcParams['axes.axisbelow'] = True


In [3]:
random.seed(2) #seed to ensure that we can get the same random values again
wind_scenarios = random.sample(range(100), 50) #How many scenarios? Sampling without replacement - taking 100 samples will just get a list from 0 to 99
wind_scenarios.sort()
#wind_scenarios

In [4]:
wind_hour = 31
n_bus = 24
gen_data = fnc.read_data('gen_data')
system_demand = fnc.read_data('system_demand')['System Demand']
load_distribution = fnc.read_data('load_distribution')
gen_data = fnc.read_data('gen_data')
gen_costs = fnc.read_data('gen_costs')[['C ($/MWh)', 'C+($/MWh)', 'C-($/MWh)']]
#line_data = fnc.read_data('line_data')
#branch_matrix = fnc.read_data('branch_matrix')
wind_data = fnc.read_data('wind_data', wind_hour=wind_hour, wind_scenarios=wind_scenarios) #The ED is only for a single hour - in this case, we choose hour 31

In [5]:
wind_data_max = wind_data.max(axis=1)
print(wind_data_max)



Wind Farm
1    145.616105
2    172.479349
3    189.099842
4    190.685053
5    115.224042
6    134.578504
dtype: float64


In [6]:
wf_costs = gen_costs.iloc[0:6].copy()
wf_costs['C ($/MWh)'] = 0.05 * gen_costs['C ($/MWh)'].mean()
wf_costs['C+($/MWh)'] = 0.1 * gen_costs['C+($/MWh)'].mean()
wf_costs['C-($/MWh)'] = 0.1 * gen_costs['C-($/MWh)'].mean()
wf_costs

,C ($/MWh),C+($/MWh),C-($/MWh)
0,0.618,1.408333,0.9
1,0.618,1.408333,0.9
2,0.618,1.408333,0.9
3,0.618,1.408333,0.9
4,0.618,1.408333,0.9
5,0.618,1.408333,0.9


In [7]:
t = 0 #hour
demand = system_demand[t]

n_gen = len(gen_data.index)
n_wf = len(wind_data.index)
n_scenarios = len(wind_scenarios)
p_max_wf = 300

In [27]:
class setting(object):
    '''
        A small class which can have attributes set
    '''
    pass

class benders_subproblem: # Class representing the subproblems for each scenario

    def __init__(self,master,scenario,P_G_master,P_W_master): # initialize class
        self.data = setting() # define data attributes
        self.variables = setting() # define variable attributes
        self.constraints = setting() # define constraints attributes
        self.master = master # define master problem to which subproblem is attached
        self._init_data(scenario,P_G_master,P_W_master) # initialize data
        self._build_model() # build gurobi model

        
    
        
    def _init_data(self,scenario,P_G_master,P_W_master): # initialize data

        self.data.scenario = scenario # add scenario
        self.data.P_G_master = P_G_master # add initial value of complicating variables
        self.data.P_W_master = P_W_master # add initial value of complicating variables
        

    def _build_model(self): # build gurobi model
        
        self.model = gb.Model(name='subproblem') # build model
        self.model.setParam('OutputFlag', 0)
        self._build_variables() # add variables
        self._build_objective() # add objective
        self._build_constraints() # add constraints
        self.model.update() 


    def _build_variables(self): # build variables

        #index shortcut 
        m = self.model
        
        # complicating variables
        self.variables.P_G = m.addVars(n_gen,lb=0,ub=gb.GRB.INFINITY,name='P_G') 
        self.variables.P_W = m.addVars(n_wf, lb=0,ub=gb.GRB.INFINITY ,name='P_W')

        self.variables.P_G_UP = m.addVars(n_gen, lb=0,ub=gb.GRB.INFINITY,name='P_G_UP') # electricity production adjustment of generators in real time (\Delta x^G_i)
        self.variables.P_G_DW = m.addVars(n_gen,lb=0,ub=gb.GRB.INFINITY,name='P_G_DW') # electricity production adjustment of generators in real time (\Delta x^G_i)
        self.variables.P_W_UP = m.addVars(n_wf, lb=0,ub=gb.GRB.INFINITY, name='P_W_UP') # electricity production adjustment of generators in real time (\Delta x^G_i)
        self.variables.P_W_DW = m.addVars(n_wf, lb=0,ub=gb.GRB.INFINITY,name='P_W_DW')# electricity production adjustment of generators in real time (\Delta x^G_i)
       
       
        m.update() # update model
    

    def _build_objective(self): # define objective function
 
        #index shortcut 
        m = self.model

        # Set the objective function for the suproblem
        subproblem_obj = gb.quicksum((gen_costs['C+($/MWh)'][g] * self.variables.P_G_UP[g] - gen_costs['C-($/MWh)'][g] * self.variables.P_G_DW[g]) for g in range(n_gen)) + gb.quicksum((wf_costs['C+($/MWh)'][w] * self.variables.P_W_UP[w] - wf_costs['C-($/MWh)'][w] * self.variables.P_W_DW[w]) for w in range(n_wf))   
        m.setObjective(subproblem_obj, gb.GRB.MINIMIZE) #minimize cost

        m.update() 
        

    def _build_constraints(self):

        #index shortcut 
        m = self.model

        self.constraints.P_G_i = m.addConstrs(self.variables.P_G[g] == self.data.P_G_master[g] for g in range(n_gen))# constraints that fix complicating variables to master problem solutions
                        
        self.constraints.P_W_i = m.addConstrs(self.variables.P_W[w] == self.data.P_W_master[w] for w in range(n_wf))# constraints that fix complicating variables to master problem solutions
                        
        #Real-time balance constraint
        self.constraints.RT_balance_constraint = m.addConstr(gb.quicksum(self.variables.P_G_UP[g] - self.variables.P_G_DW[g] for g in range(n_gen)) + 
                                                                  gb.quicksum(self.variables.P_W_UP[w] - self.variables.P_W_DW[w] for w in range(n_wf)) == 0)
        
        self.constraints.adjustment_max_generation_constraint = {} #max production of generators after adjustment
        
        #Real-time limits
        self.constraints.RT_G_min = m.addConstrs(self.variables.P_G[g] + self.variables.P_G_UP[g] - self.variables.P_G_DW[g] >= 0 for g in range(n_gen))
        self.constraints.RT_W_min = m.addConstrs(self.variables.P_W[w] + self.variables.P_W_UP[w] - self.variables.P_W_DW[w] >= 0 for w in range(n_wf))

        self.constraints.RT_G_max = m.addConstrs(self.variables.P_G[g] + self.variables.P_G_UP[g] - self.variables.P_G_DW[g] <= gen_data['P max MW'][g] for g in range(n_gen))
        self.constraints.RT_W_max = m.addConstrs(self.variables.P_W[w] + self.variables.P_W_UP[w] - self.variables.P_W_DW[w] <= p_max_wf for w in range(n_wf)) #accounting for wind scenario

        #Real-time regulating power bounds
        self.constraints.P_UP_PB = m.addConstrs(self.variables.P_G_UP[g] <= gen_data['R+ MW'][g] for g in range(n_gen))
        self.constraints.P_DW_PB = m.addConstrs(self.variables.P_G_DW[g] <= gen_data['R- MW'][g] for g in range(n_gen))

        self.constraints.W_UP_PB = m.addConstrs(self.variables.P_W_UP[w] <= p_max_wf for w in range(n_wf))
        self.constraints.W_DW_PB = m.addConstrs(self.variables.P_W_DW[w] <= p_max_wf for w in range(n_wf))
                
        m.update()


    def _update_complicating_variables(self): # function that updates the value of the complicating variables

        # index shortcut
        m = self.model

        for g in range(n_gen):
            self.constraints.P_G_i[g].rhs = self.master.variables.P_G[g].x
        for w in range(n_wf):
            self.constraints.P_W_i[w].rhs = self.master.variables.P_W[w].x
    
        m.update()



class benders_master: # class of master problem
    
    def __init__(self,epsilon,max_iters): # initialize class
        self.data = setting() # build data attributes
        self.variables = setting() # build variable attributes
        self.constraints = setting() # build contraint attributes
        self._init_data(epsilon,max_iters) # initialize data
        self._build_model() # build gurobi model
        
    
        
    def _init_data(self,epsilon,max_iters): # initialize data

        self.data.epsilon = epsilon # add value of convergence criteria
        self.data.max_iters = max_iters # add max number of iterations
        self.data.iteration = 1 # initialize value of iteration count
        self.data.upper_bounds = {} # initialize list of upper-bound values
        self.data.lower_bounds = {} # initialize list of lower-bound values
        self.data.P_G_duals = {} # initialize list of sensitivities values
        self.data.P_G_values = {} # initialize list of complicating variables values
        self.data.P_W_duals = {} # initialize list of sensitivities values
        self.data.P_W_values = {} # initialize list of complicating variables values
        self.data.gamma_values = {} # initialize list of gamma values
        self.data.subproblem_objectives = {} # initialize list of subproblems objective values
        self.data.master_objectives = {} # initialize list of master problem objective values

    def _build_model(self): # build gurobi model
        
        self.model = gb.Model(name='master') # build model
        self.model.setParam('OutputFlag', 0)
        self._build_variables() # add variables
        self._build_objective() # add objective
        self._build_constraints() # add constraints
        self.model.update()


    def _build_variables(self): # build variables

        #index shortcut 
        m = self.model
        
        # complicating variables
        self.variables.P_G = m.addVars(n_gen, lb=0, ub=gb.GRB.INFINITY, name="P_G") #Note: This is in MW
        self.variables.P_W = m.addVars(n_wf, lb=0, ub=gb.GRB.INFINITY, name="P_W") #Note: Wind farms can be curtailed

        # gamma = approximator of subproblems' objective value
        self.variables.gamma = m.addVar(lb=-1e9,name='gamma')
        m.update()
    

    def _build_objective(self): # build objective
 
        #index shortcut 
        m = self.model 

        # Set the objective function for the master problem
        master_obj = gb.quicksum(gen_costs['C+($/MWh)'][g]*self.variables.P_G[g] for g in range(n_gen)) + gb.quicksum(wf_costs['C ($/MWh)'][w] * self.variables.P_W[w] for w in range(n_wf)) + self.variables.gamma # expected electricity production cost (z)     
        m.setObjective(master_obj, gb.GRB.MINIMIZE) #minimize cost

        m.update() 

    def _build_constraints(self): # build constraints

        #index shortcut 
        m = self.model
            
       #Day-ahead constraint
        self.constraints.DA = m.addConstr(gb.quicksum(self.variables.P_G[g] for g in range(n_gen)) + gb.quicksum(self.variables.P_W[w] for w in range(n_wf)) - demand == 0)
        
        self.constraints.DA_G = m.addConstrs(self.variables.P_G[g] <= gen_data['P max MW'][g] for g in range(n_gen))
        self.constraints.DA_W = m.addConstrs(self.variables.P_W[w] <= p_max_wf for w in range(n_wf))

        self.constraints.master_cuts = {} # initialize master problem cuts (empty)
        
        m.update()


    def _build_subproblems(self): # function that builds subproblems
        
        self.subproblem = {k:benders_subproblem(master = self, scenario=k , P_G_master={g:self.variables.P_G[g].x for g in range(n_gen)},
                                                                            P_W_master={w:self.variables.P_W[w].x for w in range(n_wf)}) 
                            for k in range(n_scenarios)}
        
        

    def _update_master_cut(self): # fucntion that adds cuts to master problem

        # index shortcut
        m = self.model

        self.constraints.master_cuts[self.data.iteration] = m.addConstr(
            self.variables.gamma,
            gb.GRB.GREATER_EQUAL,
            gb.quicksum((1/n_scenarios)*(self.data.subproblem_objectives[self.data.iteration-1][k] + 
                                        (gb.quicksum(self.data.P_G_duals[self.data.iteration-1][g,k]*(self.variables.P_G[g]-
                                        self.data.P_G_values[self.data.iteration-1][g]) for g in range(n_gen))+
                                        gb.quicksum(self.data.P_W_duals[self.data.iteration-1][w,k]*(self.variables.P_W[w]-
                                        self.data.P_W_values[self.data.iteration-1][w]) for w in range(n_wf)))) for k in range(n_scenarios)), 
                                        name='new (uni)-cut at iteration {0}'.format(self.data.iteration))

        m.update()
    
    
    def _save_master_data(self): # function that saves results of master problem optimization at each iteration (complicating variables, objective value, lower bound value)
        
        # index shortcut
        m = self.model
        
        # save complicating variables value
        self.data.P_G_values[self.data.iteration] = {g:self.variables.P_G[g].x for g in range(n_gen)}
        self.data.P_W_values[self.data.iteration] = {w:self.variables.P_W[w].x for w in range(n_wf)}
        
        # save gamma value
        self.data.gamma_values[self.data.iteration] = self.variables.gamma.x
          
        # save lower bound value
        self.data.lower_bounds[self.data.iteration] = m.ObjVal

        # save master problem objective value
        self.data.master_objectives[self.data.iteration] = m.ObjVal - self.variables.gamma.x
           
        m.update()

    def _save_subproblems_data(self): # function that saves results of subproblems optimization at each iteration (sensitivities, objective value, upper bound value)
        
        # index shortcut
        m = self.model

        # save sensitivities
        self.data.P_G_duals[self.data.iteration] = {(g,k):self.subproblem[k].constraints.P_G_i[g].Pi for g in range(n_gen) for k in range(n_scenarios)}
        self.data.P_W_duals[self.data.iteration] = {(w,k):self.subproblem[k].constraints.P_W_i[w].Pi for w in range(n_wf) for k in range(n_scenarios)}

        # save subproblems objective values
        self.data.subproblem_objectives[self.data.iteration] = {k:self.subproblem[k].model.ObjVal for k in range(n_scenarios)}             
        
        # save upper bound value
        self.data.upper_bounds[self.data.iteration] = self.data.master_objectives[self.data.iteration] + sum((1/n_scenarios)*self.subproblem[k].model.ObjVal for k in range(n_scenarios))
                      
        m.update()

    def _do_benders_step(self): # function that does one benders step
        
        # index shortcut
        m = self.model

        if self.data.iteration > 1:
            self._update_master_cut() # add cut

        m.optimize() # optimize master problem
        self._save_master_data() # save master problem optimization results

        for k in range(n_scenarios): 

            if self.data.iteration > 1:
                self.subproblem[k]._update_complicating_variables() # update value of complicating constraints in subproblems

            self.subproblem[k].model.optimize() # solve subproblems

        self._save_subproblems_data() # save subproblems optimization results

               
    def _benders_iterate(self): # function that solves iteratively the benders algorithm

        # index shortcut            
        m = self.model
        m.setParam('OutputFlag', 0)
        
        # initial iteration: 
        m.optimize() #   solve master problem (1st iteration)
        self._save_master_data() # save results of master problem and lower bound
        self._build_subproblems() # build subproblems (1st iteration)
        for k in range(n_scenarios): 
            self.subproblem[k].model.optimize() # solve subproblems
        self._save_subproblems_data() # save results of subproblems and upper bound
        
        finished = False

        # do benders steps until convergence
        while not finished:
            print('\nIteration %d...' % self.data.iteration)

            if self.data.iteration > self.data.max_iters:
                finished = True
                print('Error: Maximum iterations reached.')
                break
            
            self._do_benders_step()

            if self.data.iteration > 2:
                error = abs(self.data.upper_bounds[self.data.iteration] - self.data.lower_bounds[self.data.iteration])
                print('|UB - LB| = %.2f' % error)
                
                if error < self.data.epsilon:
                    finished = True
                    print('Success: Converged at iteration %d.' % self.data.iteration)
                    break

            self.data.iteration += 1 # go to next iteration 
            
#solve and print results

DA_model = benders_master(epsilon=1e-3, max_iters=50)
DA_model._benders_iterate()

print('uni-cut optimal cost',DA_model.data.upper_bounds[DA_model.data.iteration]) # print optimal cost (last upper-bound)
print('gamma', DA_model.data.gamma_values[DA_model.data.iteration])


Iteration 1...

Iteration 2...

Iteration 3...
|UB - LB| = 0.00
Success: Converged at iteration 3.
uni-cut optimal cost 417.66603000000003
gamma 0.0


C:\Users\marku\AppData\Local\Temp\ipykernel_18152\1249656547.py:203: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  self.constraints.master_cuts[self.data.iteration] = m.addConstr(
